## Imports

In [5]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [6]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 8th Dec. 15th


In [8]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [9]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [10]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [11]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [12]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-11-2024,Condo/Co-op,1149 Hillsboro Mile #810,Hillsboro Beach,FL,33062.0,565000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1149...,Beaches MLS,F10473773,N,Y,26.292705,-80.078564


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [13]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [14]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
59,PAST SALE,2024-12-13,Condo/Co-op,1124 Seminole Dr Unit 3A,Fort Lauderdale,FL,33304.0,305000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1124...,Beaches MLS,F10463137,N,Y,26.140607,-80.110956
94,PAST SALE,2024-12-13,Condo/Co-op,2101 Atlantic Shores Blvd #319,Hallandale,FL,33009.0,222000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/210...,Beaches MLS,F10448855,N,Y,25.995837,-80.127698
23,PAST SALE,2024-12-13,Condo/Co-op,801 S Federal Hwy #1003,Pompano Beach,FL,33062.0,277500.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-S-...,Beaches MLS,F10466230,N,Y,26.220577,-80.105540
25,PAST SALE,2024-12-13,Condo/Co-op,9470 Poinciana Pl #309,Davie,FL,33324.0,209000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Davie/9470-Poinciana...,Beaches MLS,F10447153,N,Y,26.097347,-80.274666
26,PAST SALE,2024-12-13,Condo/Co-op,9551 Weldon Cir Unit E-406,Fort Lauderdale,FL,33321.0,223000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/9551...,Beaches MLS,F10469471,N,Y,26.219258,-80.277413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,PAST SALE,2024-12-09,Condo/Co-op,10456 NW 24th Pl #301,Sunrise,FL,33322.0,150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/10456-NW-24t...,MARMLS,A11635633,N,Y,26.156337,-80.289091
76,PAST SALE,2024-12-09,Condo/Co-op,2124 NE 5th Ave #110,Wilton Manors,FL,33305.0,267500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Wilton-Manors/2124-N...,Beaches MLS,F10453213,N,Y,26.155660,-80.139850
30,PAST SALE,2024-12-09,Condo/Co-op,12750 SW 15th St Unit 308D,Pembroke Pines,FL,33027.0,179900.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/12750...,MARMLS,A11682577,N,Y,25.994780,-80.317040
82,PAST SALE,2024-12-09,Condo/Co-op,711 N Riverside Dr #103,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/711-N-...,Beaches MLS,RX-11012410,N,Y,26.243316,-80.090814


In [15]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [16]:
df_top_ten = df.head(10)

## Collect Agent Information

In [17]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [18]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [19]:
response_df = pd.DataFrame(response_list)

In [20]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [21]:
df_top_ten = merged_df

## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

117


In [23]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$312,735


In [24]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$432


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$36,590,015


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [26]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
154
Input Previous Week Condo Average Sales Price:
379013
Input Previous Week Condo Average PSF:
287
Input Previous Week Condo Sales total (ex: 198_000_000)
32_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [27]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [28]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [29]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [30]:
pd.set_option('display.max_columns',None)

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange


In [32]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange
1,PAST SALE,2024-12-10,Condo/Co-op,20 Isle Of Venice Dr #302,Fort Lauderdale,FL,33301.0,1875000.0,3.0,2.0,AQUALUNA LAS OLAS CONDO,2590.0,NaN,2016.0,NaN,724.0,1829.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,MARMLS,A11573273,N,Y,26.121158,-80.120244,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,2,blue
2,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,3,blue
3,PAST SALE,2024-12-09,Condo/Co-op,1440 S Ocean Blvd Unit 4B,Lauderdale By The Sea,FL,33062.0,1070000.0,3.0,3.0,CORNICHE CONDO,3000.0,NaN,1991.0,NaN,357.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1440-S...,MARMLS,A11534357,N,Y,26.213759,-80.091760,https://www.redfin.com/FL/Pompano-Beach/1440-S...,Bonnie Staskowski,Coldwell Banker Realty,Carey Johnson,Coldwell Banker Realty,Joann Manning,Reset Real Estate,None,None,4,blue
4,PAST SALE,2024-12-12,Condo/Co-op,2821 N Ocean Blvd Unit 802S,Fort Lauderdale,FL,33308.0,830000.0,2.0,2.0,Sapphire Fort Lauderdale,1274.0,NaN,2009.0,NaN,651.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10426315,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Theodore Wootton,"Compass Florida, LLC",None,None,5,blue
5,PAST SALE,2024-12-13,Condo/Co-op,1815 N Surf Rd #304,Hollywood,FL,33019.0,725000.0,2.0,2.0,LA PLAYA CONDOMINIUMS,827.0,NaN,1972.0,NaN,877.0,2006.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,MARMLS,A11686548,N,Y,26.023391,-80.115156,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,Angella Arias,Brickss Real Estate Company,None,None,Stephanie Jimenez,EXP Realty LLC,None,None,6,blue
6,PAST SALE,2024-12-13,Condo/Co-op,2900 NE 14th St Cswy #807,Pompano Beach,FL,33062.0,650000.0,2.0,2.0,Voyager Condo,1450.0,NaN,1981.0,NaN,448.0,1269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Beaches MLS,RX-11020973,N,Y,26.249652,-80.091467,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Mary Mangines,Compass Florida LLC,Bonnie Heatzig,Compass Florida LLC,Denise Stewart,Keller Williams Realty Profess,None,None,7,blue
7,PAST SALE,2024-12-13,Condo/Co-op,2500 E Las Olas Blvd #1708,Fort Lauderdale,FL,33301.0,643000.0,2.0,2.0,Marine Tower,1800.0,NaN,1969.0,NaN,357.0,1692.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2500...,Beaches MLS,F10459256,N,Y,26.119634,-80.112128,https://www.redfin.com/FL/Fort-Lauderdale/2500...,Debra Giglia,RE/MAX Preferred,None,None,William Burkhart,Casa Bella Properties Inc,None,None,8,blue
8,PAST SALE,2024-12-09,Condo/Co-op,4010 S Ocean Dr Unit R2506,Hollywood,FL,33019.0,585000.0,1.

In [34]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Palm Beach County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [35]:
m.save('index.html')

## Data snagger

In [36]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [37]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [38]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_12162024


# CREATE TEMPLATE 

In [39]:
muni_set = set(df_top_ten['CITY'])

In [40]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [41]:
df_top_ten.reset_index(inplace=True,drop=True)

In [42]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [43]:
top_sale

'RIVA at 1180 N Federal Hwy #801 in Fort Lauderdale'

In [44]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange,RIVA at 1180 N Federal Hwy #801 in Fort Lauder...
1,PAST SALE,2024-12-10,Condo/Co-op,20 Isle Of Venice Dr #302,Fort Lauderdale,FL,33301.0,1875000.0,3.0,2.0,AQUALUNA LAS OLAS CONDO,2590.0,NaN,2016.0,NaN,724.0,1829.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,MARMLS,A11573273,N,Y,26.121158,-80.120244,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,2,blue,AQUALUNA LAS OLAS CONDO at 20 Isle Of Venice D...
2,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,3,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
3,PAST SALE,2024-12-09,Condo/Co-op,1440 S Ocean Blvd Unit 4B,Lauderdale By The Sea,FL,33062.0,1070000.0,3.0,3.0,CORNICHE CONDO,3000.0,NaN,1991.0,NaN,357.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1440-S...,MARMLS,A11534357,N,Y,26.213759,-80.091760,https://www.redfin.com/FL/Pompano-Beach/1440-S...,Bonnie Staskowski,Coldwell Banker Realty,Carey Johnson,Coldwell Banker Realty,Joann Manning,Reset Real Estate,None,None,4,blue,CORNICHE CONDO at 1440 S Ocean Blvd Unit 4B in...
4,PAST SALE,2024-12-12,Condo/Co-op,2821 N Ocean Blvd Unit 802S,Fort Lauderdale,FL,33308.0,830000.0,2.0,2.0,Sapphire Fort Lauderdale,1274.0,NaN,2009.0,NaN,651.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10426315,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Theodore Wootton,"Compass Florida, LLC",None,None,5,blue,Sapphire Fort Lauderdale at 2821 N Ocean Blvd ...
5,PAST SALE,2024-12-13,Condo/Co-op,1815 N Surf Rd #304,Hollywood,FL,33019.0,725000.0,2.0,2.0,LA PLAYA CONDOMINIUMS,827.0,NaN,1972.0,NaN,877.0,2006.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,MARMLS,A11686548,N,Y,26.023391,-80.115156,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,Angella Arias,Brickss Real Estate Company,None,None,Stephanie Jimenez,EXP Realty LLC,None,None,6,blue,LA PLAYA CONDOMINIUMS at 1815 N Surf Rd #304 i...
6,PAST SALE,2024-12-13,Condo/Co-op,2900 NE 14th St Cswy #807,Pompano Beach,FL,33062.0,650000.0,2.0,2.0,Voyager Condo,1450.0,NaN,1981.0,NaN,448.0,1269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Beaches MLS,RX-11020973,N,Y,26.249652,-80.091467,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Mary Mangines,Compass Florida LLC,Bonnie Heatzig,Compass Florida LLC,Denise Stewart,Keller Williams Realty Profess,None,None,7,blue,Voyager Condo at 2900 NE 14th St Cswy #807 in ...
7,PAST SALE,2024-12-13,Condo/Co-op,2500 E Las Olas Blvd #1708,Fort Lauderdale,FL,33301.0,643000.0,2.0,2.0,Marine Tower,1800.0,NaN,1969.0,NaN,357.0,1

In [47]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: RIVA closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $575,000M to $2,050,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Fort Lauderdale, Pompano Beach, Lauderdale By The Sea

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 117 condo sales totaling $36,590,015 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 154 condo sales totaling $32,300,000.

Last week’s units sold for an average of $312,735, lower than the $379,013 average price from the previous week. The average price per square

In [48]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [50]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [51]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange,RIVA at 1180 N Federal Hwy #801 in Fort Lauder...
1,PAST SALE,2024-12-10,Condo/Co-op,20 Isle Of Venice Dr #302,Fort Lauderdale,FL,33301.0,1875000.0,3.0,2.0,AQUALUNA LAS OLAS CONDO,2590.0,NaN,2016.0,NaN,724.0,1829.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,MARMLS,A11573273,N,Y,26.121158,-80.120244,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,2,blue,AQUALUNA LAS OLAS CONDO at 20 Isle Of Venice D...
2,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,3,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
3,PAST SALE,2024-12-09,Condo/Co-op,1440 S Ocean Blvd Unit 4B,Lauderdale By The Sea,FL,33062.0,1070000.0,3.0,3.0,CORNICHE CONDO,3000.0,NaN,1991.0,NaN,357.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1440-S...,MARMLS,A11534357,N,Y,26.213759,-80.091760,https://www.redfin.com/FL/Pompano-Beach/1440-S...,Bonnie Staskowski,Coldwell Banker Realty,Carey Johnson,Coldwell Banker Realty,Joann Manning,Reset Real Estate,None,None,4,blue,CORNICHE CONDO at 1440 S Ocean Blvd Unit 4B in...
4,PAST SALE,2024-12-12,Condo/Co-op,2821 N Ocean Blvd Unit 802S,Fort Lauderdale,FL,33308.0,830000.0,2.0,2.0,Sapphire Fort Lauderdale,1274.0,NaN,2009.0,NaN,651.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10426315,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Theodore Wootton,"Compass Florida, LLC",None,None,5,blue,Sapphire Fort Lauderdale at 2821 N Ocean Blvd ...
5,PAST SALE,2024-12-13,Condo/Co-op,1815 N Surf Rd #304,Hollywood,FL,33019.0,725000.0,2.0,2.0,LA PLAYA CONDOMINIUMS,827.0,NaN,1972.0,NaN,877.0,2006.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,MARMLS,A11686548,N,Y,26.023391,-80.115156,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,Angella Arias,Brickss Real Estate Company,None,None,Stephanie Jimenez,EXP Realty LLC,None,None,6,blue,LA PLAYA CONDOMINIUMS at 1815 N Surf Rd #304 i...
6,PAST SALE,2024-12-13,Condo/Co-op,2900 NE 14th St Cswy #807,Pompano Beach,FL,33062.0,650000.0,2.0,2.0,Voyager Condo,1450.0,NaN,1981.0,NaN,448.0,1269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Beaches MLS,RX-11020973,N,Y,26.249652,-80.091467,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Mary Mangines,Compass Florida LLC,Bonnie Heatzig,Compass Florida LLC,Denise Stewart,Keller Williams Realty Profess,None,None,7,blue,Voyager Condo at 2900 NE 14th St Cswy #807 in ...
7,PAST SALE,2024-12-13,Condo/Co-op,2500 E Las Olas Blvd #1708,Fort Lauderdale,FL,33301.0,643000.0,2.0,2.0,Marine Tower,1800.0,NaN,1969.0,NaN,357.0,1

In [52]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: RIVA closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $575,000M to $2,050,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Fort Lauderdale, Pompano Beach, Lauderdale By The Sea

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 117 condo sales totaling $36,590,015 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 154 condo sales totaling $32,300,000.

Last week’s units sold for an average of $312,735, lower than the $379,013 average price from the previous week. The average price per square

In [53]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/1180-N-Federal-Hwy-33304/unit-801/home/63058593


In [54]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/20-Isle-of-Venice-Dr-33301/unit-302/home/113134140


In [55]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hollywood/2301-S-Ocean-Dr-33019/unit-2004/home/41953809


In [56]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hollywood/4010-S-Ocean-Dr-33019/unit-R2506/home/172066913


In [57]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Hollywood/1815-N-Surf-Rd-33019/unit-304/home/41930764


In [58]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/1440-S-Ocean-Blvd-33062/unit-4B/home/41701245


In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-11,Condo/Co-op,1180 N Federal Hwy #801,Fort Lauderdale,FL,33304.0,2050000.0,3.0,3.0,RIVA,2407.0,NaN,2018.0,NaN,852.0,2243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1180...,MARMLS,A11681185,N,Y,26.139647,-80.118966,https://www.redfin.com/FL/Fort-Lauderdale/1180...,Jonathan Howard,Douglas Elliman,Melanie Kimpton,Douglas Elliman,None,None,None,None,1,orange,RIVA at 1180 N Federal Hwy #801 in Fort Lauder...
1,PAST SALE,2024-12-10,Condo/Co-op,20 Isle Of Venice Dr #302,Fort Lauderdale,FL,33301.0,1875000.0,3.0,2.0,AQUALUNA LAS OLAS CONDO,2590.0,NaN,2016.0,NaN,724.0,1829.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,MARMLS,A11573273,N,Y,26.121158,-80.120244,https://www.redfin.com/FL/Fort-Lauderdale/20-I...,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,Sherri Pfefer,Coldwell Banker Realty,Megan Pfefer,Coldwell Banker Realty,2,blue,AQUALUNA LAS OLAS CONDO at 20 Isle Of Venice D...
2,PAST SALE,2024-12-09,Condo/Co-op,411 N New River Dr #1406,Fort Lauderdale,FL,33301.0,1650000.0,3.0,3.5,Las Olas Grand,2635.0,NaN,2005.0,NaN,626.0,2884.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10428680,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Dennis Miller,Better Homes & Gdns RE Fla 1st,None,None,Sharon O'Connor,Coldwell Banker Realty,None,None,3,blue,Las Olas Grand at 411 N New River Dr #1406 in ...
3,PAST SALE,2024-12-09,Condo/Co-op,1440 S Ocean Blvd Unit 4B,Lauderdale By The Sea,FL,33062.0,1070000.0,3.0,3.0,CORNICHE CONDO,3000.0,NaN,1991.0,NaN,357.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1440-S...,MARMLS,A11534357,N,Y,26.213759,-80.091760,https://www.redfin.com/FL/Pompano-Beach/1440-S...,Bonnie Staskowski,Coldwell Banker Realty,Carey Johnson,Coldwell Banker Realty,Joann Manning,Reset Real Estate,None,None,4,blue,CORNICHE CONDO at 1440 S Ocean Blvd Unit 4B in...
4,PAST SALE,2024-12-12,Condo/Co-op,2821 N Ocean Blvd Unit 802S,Fort Lauderdale,FL,33308.0,830000.0,2.0,2.0,Sapphire Fort Lauderdale,1274.0,NaN,2009.0,NaN,651.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10426315,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Theodore Wootton,"Compass Florida, LLC",None,None,5,blue,Sapphire Fort Lauderdale at 2821 N Ocean Blvd ...
5,PAST SALE,2024-12-13,Condo/Co-op,1815 N Surf Rd #304,Hollywood,FL,33019.0,725000.0,2.0,2.0,LA PLAYA CONDOMINIUMS,827.0,NaN,1972.0,NaN,877.0,2006.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,MARMLS,A11686548,N,Y,26.023391,-80.115156,https://www.redfin.com/FL/Hollywood/1815-N-Sur...,Angella Arias,Brickss Real Estate Company,None,None,Stephanie Jimenez,EXP Realty LLC,None,None,6,blue,LA PLAYA CONDOMINIUMS at 1815 N Surf Rd #304 i...
6,PAST SALE,2024-12-13,Condo/Co-op,2900 NE 14th St Cswy #807,Pompano Beach,FL,33062.0,650000.0,2.0,2.0,Voyager Condo,1450.0,NaN,1981.0,NaN,448.0,1269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Beaches MLS,RX-11020973,N,Y,26.249652,-80.091467,https://www.redfin.com/FL/Pompano-Beach/2900-N...,Mary Mangines,Compass Florida LLC,Bonnie Heatzig,Compass Florida LLC,Denise Stewart,Keller Williams Realty Profess,None,None,7,blue,Voyager Condo at 2900 NE 14th St Cswy #807 in ...
7,PAST SALE,2024-12-13,Condo/Co-op,2500 E Las Olas Blvd #1708,Fort Lauderdale,FL,33301.0,643000.0,2.0,2.0,Marine Tower,1800.0,NaN,1969.0,NaN,357.0,1

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 4, 26) ## List (Earlier) date
date2 = datetime(2024, 12, 10) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

228


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [69]:
csv_date_string = today.strftime("%m_%d_%Y")

In [70]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [71]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-11,1180 N Federal Hwy #801 in Fort Lauderdale,"$2,050,000",3,3.0,"2,407",2018,$852,MARMLS,A11681185,Jonathan Howard Douglas Elliman Melanie Kimpto...,
1,2024-12-10,20 Isle Of Venice Dr #302 in Fort Lauderdale,"$1,875,000",3,2.0,"2,590",2016,$724,MARMLS,A11573273,Sherri Pfefer Coldwell Banker Realty Megan Pfe...,Sherri Pfefer Coldwell Banker Realty Megan Pfe...
2,2024-12-09,411 N New River Dr #1406 in Fort Lauderdale,"$1,650,000",3,3.5,"2,635",2005,$626,Beaches MLS,F10428680,Dennis Miller Better Homes & Gdns RE Fla 1st,Sharon O'Connor Coldwell Banker Realty
3,2024-12-09,1440 S Ocean Blvd Unit 4B in Lauderdale By The...,"$1,070,000",3,3.0,"3,000",1991,$357,MARMLS,A11534357,Bonnie Staskowski Coldwell Banker Realty Carey...,Joann Manning Reset Real Estate
4,2024-12-12,2821 N Ocean Blvd Unit 802S in Fort Lauderdale,"$830,000",2,2.0,"1,274",2009,$651,Beaches MLS,F10426315,David Saindon RE/MAX Experience Robert Shastan...,"Theodore Wootton Compass Florida, LLC"
5,2024-12-13,1815 N Surf Rd #304 in Hollywood,"$725,000",2,2.0,827,1972,$877,MARMLS,A11686548,Angella Arias Brickss Real Estate Company,Stephanie Jimenez EXP Realty LLC
6,2024-12-13,2900 NE 14th St Cswy #807 in Pompano Beach,"$650,000",2,2.0,"1,450",1981,$448,Beaches MLS,RX-11020973,Mary Mangines Compass Florida LLC Bonnie Heatz...,Denise Stewart Keller Williams Realty Profess
7,2024-12-13,2500 E Las Olas Blvd #1708 in Fort Lauderdale,"$643,000",2,2.0,"1,800",1969,$357,Beaches MLS,F10459256,Debra Giglia RE/MAX Preferred,William Burkhart Casa Bella Properties Inc
8,2024-12-09,4010 S Ocean Dr Unit R2506 in Hollywood,"$585,000",1,1.5,702,2019,$833,MARMLS,A11540926,Antonio Valentino Aval Realty LLC,Tibizay Morales Gables Gate Enterprises Inc.
9,2024-12-10,2301 S Ocean Dr #2004 in Hollywood,"$575,000",2,2.0,"1,070",1978,$537,MARMLS,A11670169,"Svetlana Perez LP Global Realty, LLC Mariya Vl...",
